In [2]:
from crystal_functions.file_readwrite import *
from crystal_functions.convert import *
import numpy as np
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import *
import phonopy
from os.path import join
import shutil

import os
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor

## pymatgen interface

In [5]:
from pymatgen.io.phonopy import get_displaced_structures
mno_gui_opt = Crystal_gui().read_cry_gui('./data/MnO.gui')
mno_pmg_opt = cry_gui2pmg(mno_gui_opt)
#mno_pmg.replace(2,1)
#mno_pmg.add_spin_by_site([+4,0,-4,0])
#mno_pmg.add_oxidation_state_by_site([+4,0,-4,0])

supercell_matrix = np.identity(3)*6
supercell_matrix = np.identity(3)*2

disp_str = get_displaced_structures(mno_pmg_opt,atom_disp=0.01, supercell_matrix=supercell_matrix)
mno_scel_atomic_numbers = disp_str[0].atomic_numbers

In [83]:
mno_pmg_opt

Structure Summary
Lattice
    abc : 5.3151159200021905 5.315115920000355 5.315115920000355
 angles : 34.22791099990832 34.22791099998824 34.22791099998824
 volume : 42.36314758432684
      A : 1.80606218932 0.0 4.99885953107
      B : -0.903031094659 1.56409573676 4.99885953107
      C : -0.903031094659 -1.56409573676 4.99885953107
PeriodicSite: Mn (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: O (-1.8061, -0.0000, -1.2497) [-0.7500, 0.2500, 0.2500]
PeriodicSite: Mn (-0.9030, -1.5641, -2.4994) [-0.5000, -0.5000, 0.5000]
PeriodicSite: O (-0.9030, -1.5641, 1.2497) [-0.2500, -0.2500, 0.7500]

In [39]:
new_lattice_2 = np.matmul(qq,mno_pmg_opt.lattice.matrix)

In [21]:
new_lattice_1 = np.matmul(conv_matrix,mno_pmg_opt.lattice.matrix)

In [40]:
Structure(new_lattice_2,[],[])

Structure Summary
Lattice
    abc : 4.392560947738861 4.39256094773121 4.39256094773121
 angles : 90.821316026348 90.8213160265301 90.8213160265301
 volume : 84.72629516865366
      A : 3.612124378639 0.0 2.499429765535
      B : -1.8060621893189999 3.12819147352 2.499429765535
      C : -1.8060621893189999 -3.12819147352 2.499429765535

In [31]:
conv_matrix = np.array([[-1, 1, 1],[1, -1, 1],[1,1,-1]])
q = np.matmul(conv_matrix,conv_matrix)
q

array([[ 3, -1, -1],
       [-1,  3, -1],
       [-1, -1,  3]])

In [34]:
qq = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
qq

array([[ 1.5, -0.5, -0.5],
       [-0.5,  1.5, -0.5],
       [-0.5, -0.5,  1.5]])

In [3]:
def get_cell( a, b, c, alpha, beta, gamma ):
    toDeg = 180.0/np.pi
    toRad = np.pi/180.0
    cell = np.zeros( (3,3), dtype=np.float64 )
    if alpha != 90.0:
       cosa, cosb = np.cos( alpha*toRad ), np.cos( beta*toRad )
    else:
       cosa, cosb = 0.0, 0.0
    if gamma != 90.0:
       sing, cosg = np.sin( gamma*toRad ), np.cos( gamma*toRad )
    else:
       sing, cosg = 1.0, 0.0
    cell[1][0], cell[2][0], cell[2][1] = 0.0, 0.0, 0.0
    cell[0][0] = a
    cell[1][0] = b*cosg
    cell[1][1] = b*sing
    cell[2][0] = c*cosb
    cell[2][1] = c*(cosa - cosg*cosb)/sing
    trm1 = cell[2][1]/c
    cell[2][2] = c*np.sqrt( 1.0 - np.power(cosb,2) - np.power(trm1, 2) )
    return cell

In [4]:
a,b,c = mno_pmg_opt.lattice.abc
alpha, beta, gamma = mno_pmg_opt.lattice.angles

NameError: name 'mno_pmg_opt' is not defined

In [298]:
# ANGLE WORKS (DO NOT MODIFY!!!)
delta_angle = 1.
shift = 10.
to_rad = np.pi/180
to_deg = 180/np.pi

conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
conv_matrix_inv = np.linalg.inv(conv_matrix)
cubic_lattice = np.matmul(conv_matrix,mno_pmg_opt.lattice.matrix)



A = cubic_lattice[0,:]
B = cubic_lattice[1,:]
C = cubic_lattice[2,:]

len_A = np.linalg.norm(cubic_lattice[0,:])
len_B = np.linalg.norm(cubic_lattice[1,:])
len_C = np.linalg.norm(cubic_lattice[2,:])

A_norm = A/len_A
B_norm = B/len_B
C_norm = C/len_C

alpha = np.arccos(np.dot(B_norm,C_norm))*(180/np.pi)
beta = np.arccos(np.dot(A_norm,C_norm))*(180/np.pi)
gamma = np.arccos(np.dot(A_norm,B_norm))*(180/np.pi)

###
cubic_lattice_new = np.zeros((3,3))
cubic_lattice_new[0,:] = (cubic_lattice[0,:]/len_A) * (len_A+shift)
cubic_lattice_new[1,:] = (cubic_lattice[1,:]/len_B) * (len_B+shift)
cubic_lattice_new[2,:] = (cubic_lattice[2,:]/len_C) * (len_C+shift)

prim_lattice_new = np.matmul(conv_matrix_inv,cubic_lattice_new)

A_prim_new = prim_lattice_new[0,:]
B_prim_new = prim_lattice_new[1,:]
C_prim_new = prim_lattice_new[2,:]

len_A_prim = np.linalg.norm(A_prim_new)
len_B_prim = np.linalg.norm(B_prim_new)
len_C_prim = np.linalg.norm(C_prim_new)


vector_sum = A_norm + B_norm + C_norm
len_vector_sum = np.linalg.norm(vector_sum)

vector_sum_norm = vector_sum/len_vector_sum

D_a = np.cross(np.cross(A_norm,vector_sum_norm),A_norm)
D_b = np.cross(np.cross(B_norm,vector_sum_norm),B_norm)
D_c = np.cross(np.cross(C_norm,vector_sum_norm),C_norm)

len_D_a = np.linalg.norm(D_a)
len_D_b = np.linalg.norm(D_b)
len_D_c = np.linalg.norm(D_c)

D_a_norm = D_a/len_D_a
D_b_norm = D_b/len_D_b
D_c_norm = D_c/len_D_c

alpha_t = np.arccos(np.dot(A_norm,vector_sum_norm))*(180/np.pi)
beta_t = np.arccos(np.dot(B_norm,vector_sum_norm))*(180/np.pi)
gamma_t = np.arccos(np.dot(C_norm,vector_sum_norm))*(180/np.pi)

len_vector_sum_p_tmp  = np.sqrt((3+6*(np.cos((alpha+delta_angle)*to_rad))))
len_vector_sum_p = np.sqrt(3 + 2*(np.cos((alpha+delta_angle)*to_rad) +
                                    np.cos((beta+delta_angle)*to_rad) +
                                    np.cos((gamma+delta_angle)*to_rad)))


alpha_p = (np.arccos((  1+ np.cos((beta+delta_angle)*to_rad) + np.cos((gamma+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg
beta_p = (np.arccos((  1+ np.cos((alpha+delta_angle)*to_rad) + np.cos((gamma+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg
gamma_p = (np.arccos((  1+ np.cos((alpha+delta_angle)*to_rad) + np.cos((beta+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg


delta_alpha_t = (alpha_t-alpha_p)*to_rad
delta_beta_t = (beta_t-beta_p)*to_rad
delta_gamma_t = (gamma_t-gamma_p)*to_rad

#print(alpha,alpha_t,alpha_p,delta_alpha_t)

A1 = (np.cos(delta_alpha_t)*A_norm + np.sin(delta_alpha_t)*D_a_norm) #* len_A
B1 = (np.cos(delta_beta_t)*B_norm + np.sin(delta_beta_t)*D_b_norm) #* len_B
C1 = (np.cos(delta_gamma_t)*C_norm + np.sin(delta_gamma_t)*D_c_norm) #* len_C

new_lattice_conv = np.array([A1,B1,C1])

new_lattice_conv[0,:] = new_lattice_conv[0,:] *(len_A+shift)
new_lattice_conv[1,:] = new_lattice_conv[1,:] *(len_B+shift)
new_lattice_conv[2,:] = new_lattice_conv[2,:] *(len_C+shift)

new_lattice_prim = np.matmul(conv_matrix_inv,new_lattice_conv)


display(Structure(np.matmul(conv_matrix,mno_pmg_opt.lattice.matrix),[],[]))
Structure(np.matmul(conv_matrix,new_lattice_prim),[],[])


Structure Summary
Lattice
    abc : 4.392560947738861 4.39256094773121 4.39256094773121
 angles : 90.821316026348 90.8213160265301 90.8213160265301
 volume : 84.72629516865366
      A : 3.612124378639 0.0 2.499429765535
      B : -1.8060621893189999 3.12819147352 2.499429765535
      C : -1.8060621893189999 -3.12819147352 2.499429765535

Structure Summary
Lattice
    abc : 14.392560947738867 14.39256094773121 14.39256094773121
 angles : 91.82131602634696 91.82131602653064 91.82131602653064
 volume : 2976.742056042601
      A : 11.936762396532139 0.0 8.041113985208405
      B : -5.968381198271569 10.337539474313758 8.041113985218765
      C : -5.968381198271569 -10.337539474313758 8.041113985218765

In [ ]:
Str

In [250]:
angle = 1.
shift = 0.
to_rad = np.pi/180
angle = angle*to_rad

#CELL

#Conversion matrices
conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
conv_matrix_inv = np.linalg.inv(conv_matrix)
cubic_lattice = np.matmul(conv_matrix,mno_pmg_opt.lattice.matrix)

#New cubic lattice

len_A = np.linalg.norm(cubic_lattice[0,:])
len_B = np.linalg.norm(cubic_lattice[1,:])
len_C = np.linalg.norm(cubic_lattice[2,:])

cubic_lattice[0,:] = (cubic_lattice[0,:]/len_A) * (len_A+shift)
cubic_lattice[1,:] = (cubic_lattice[1,:]/len_B) * (len_B+shift)
cubic_lattice[2,:] = (cubic_lattice[2,:]/len_C) * (len_C+shift)
print(cubic_lattice)

#New prim lattice
prim_lattice = np.matmul(conv_matrix_inv,cubic_lattice)

A_prim = prim_lattice[0,:]
B_prim = prim_lattice[1,:]
C_prim = prim_lattice[2,:]

len_A_prim = np.linalg.norm(A_prim)
len_B_prim = np.linalg.norm(B_prim)
len_C_prim = np.linalg.norm(C_prim)


A = cubic_lattice[0,:]
B = cubic_lattice[1,:]
C = cubic_lattice[2,:]

len_A = np.linalg.norm(A)
len_B = np.linalg.norm(B)
len_C = np.linalg.norm(C)


A_norm = A/len_A
B_norm = B/len_B
C_norm = C/len_C

alpha = np.arccos(np.dot(B_norm,C_norm))*(180/np.pi)
beta = np.arccos(np.dot(A_norm,C_norm))*(180/np.pi)
gamma = np.arccos(np.dot(A_norm,B_norm))*(180/np.pi)

vector_sum = A_norm+B_norm+C_norm

len_vector_sum = np.linalg.norm(vector_sum)
vector_sum_norm = vector_sum/len_vector_sum

D_a = np.cross(np.cross(A_norm,vector_sum_norm),A_norm)
D_b = np.cross(np.cross(B_norm,vector_sum_norm),B_norm)
D_c = np.cross(np.cross(C_norm,vector_sum_norm),C_norm)

len_D_a = np.linalg.norm(D_a)
len_D_b = np.linalg.norm(D_b)
len_D_c = np.linalg.norm(D_c)

D_a_norm = D_a/len_D_a
D_b_norm = D_b/len_D_b
D_c_norm = D_c/len_D_c

alpha_t = np.arccos(np.dot(A_norm,vector_sum_norm))*(180/np.pi)
beta_t = np.arccos(np.dot(B_norm,vector_sum_norm))*(180/np.pi)
gamma_t = np.arccos(np.dot(C_norm,vector_sum_norm))*(180/np.pi)

len_vector_sum_p = np.sqrt(3 + 2*(np.cos((alpha+delta_angle)*to_rad) +
                                    np.cos((beta+delta_angle)*to_rad) +
                                    np.cos((gamma+delta_angle)*to_rad)))

alpha_p = (np.arccos((  1+ np.cos((beta+delta_angle)*to_rad) + np.cos((gamma+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg
beta_p = (np.arccos((  1+ np.cos((alpha+delta_angle)*to_rad) + np.cos((gamma+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg
gamma_p = (np.arccos((  1+ np.cos((alpha+delta_angle)*to_rad) + np.cos((beta+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg

delta_alpha_t = (alpha_t-alpha_p)*to_rad
delta_beta_t = (beta_t-beta_p)*to_rad
delta_gamma_t = (gamma_t-gamma_p)*to_rad

A1 = (np.cos(angle)*A_norm + np.sin(angle)*D_a_norm) #* len_A
B1 = (np.cos(angle)*B_norm + np.sin(angle)*D_b_norm) #* len_B
C1 = (np.cos(angle)*C_norm + np.sin(angle)*D_c_norm) #* len_C

new_lattice_conv = np.array([A1,B1,C1])


new_lattice_prim = np.matmul(conv_matrix_inv,new_lattice_conv)

len_A_prim_new = np.linalg.norm(new_lattice_prim[0,:])
len_B_prim_new = np.linalg.norm(new_lattice_prim[1,:])
len_C_prim_new = np.linalg.norm(new_lattice_prim[2,:])


new_lattice_prim[0,:] = (new_lattice_prim[0,:]/ len_A_prim_new) * (len_A_prim)
new_lattice_prim[1,:] = (new_lattice_prim[1,:]/len_B_prim_new) * (len_B_prim)
new_lattice_prim[2,:] = (new_lattice_prim[2,:]/len_C_prim_new)  *(len_C_prim)

print(np.linalg.norm(new_lattice_prim[0,:]))
print(np.matmul(conv_matrix,new_lattice_prim))
aa = Structure(new_lattice,mno_pmg_opt.atomic_numbers,mno_pmg_opt.frac_coords)
Structure(np.matmul(conv_matrix,new_lattice_prim),[],[])

[[ 3.61212438  0.          2.49942977]
 [-1.80606219  3.12819147  2.49942977]
 [-1.80606219 -3.12819147  2.49942977]]
5.315115920002191
[[ 3.49514025  0.          2.50980357]
 [-1.74757012  3.02688024  2.50980357]
 [-1.74757012 -3.02688024  2.50980357]]


Structure Summary
Lattice
    abc : 4.302919860799546 4.30291986079296 4.30291986079296
 angles : 89.40858791294572 89.40858791308888 89.40858791308888
 volume : 79.65642917248634
      A : 3.4951402478580524 0.0 2.5098035732435178
      B : -1.7475701239312018 3.026880244427586 2.509803573239066
      C : -1.7475701239312018 -3.026880244427586 2.509803573239066

In [252]:
Structure(np.matmul(conv_matrix,mno_pmg_opt.lattice.matrix),[],[])

Structure Summary
Lattice
    abc : 4.392560947738861 4.39256094773121 4.39256094773121
 angles : 90.821316026348 90.8213160265301 90.8213160265301
 volume : 84.72629516865366
      A : 3.612124378639 0.0 2.499429765535
      B : -1.8060621893189999 3.12819147352 2.499429765535
      C : -1.8060621893189999 -3.12819147352 2.499429765535

In [33]:
angle = 1.
shift = 0.
to_rad = np.pi/180
angle = angle*to_rad

#CELL
conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
conv_matrix_inv = np.linalg.inv(conv_matrix)
cubic_lattice = np.matmul(conv_matrix,mno_pmg_opt.lattice.matrix)

#Original cubic lattice
A = cubic_lattice[0,:]
B = cubic_lattice[1,:]
C = cubic_lattice[2,:]

len_A = np.linalg.norm(A)
len_B = np.linalg.norm(B)
len_C = np.linalg.norm(C)

A_norm = A/len_A
B_norm = B/len_B
C_norm = C/len_C

#New cubic lattice
cubic_lattice_new = np.zeros((3,3))

cubic_lattice_new[0,:] = (cubic_lattice[0,:]/len_A) * (len_A+shift)
cubic_lattice_new[1,:] = (cubic_lattice[1,:]/len_B) * (len_B+shift)
cubic_lattice_new[2,:] = (cubic_lattice[2,:]/len_C) * (len_C+shift)

prim_lattice_new = np.matmul(conv_matrix_inv,cubic_lattice_new)

A_prim_new = prim_lattice_new[0,:]
B_prim_new = prim_lattice_new[1,:]
C_prim_new = prim_lattice_new[2,:]

#New primitive vector lengths
len_A_prim = np.linalg.norm(A_prim_new)
len_B_prim = np.linalg.norm(B_prim_new)
len_C_prim = np.linalg.norm(C_prim_new)

#Central direction
vector_sum = A+B+C

len_vector_sum = np.linalg.norm(vector_sum)
vector_sum_norm = vector_sum/len_vector_sum

D_a = np.cross(np.cross(A_norm,vector_sum_norm),A_norm)
D_b = np.cross(np.cross(B_norm,vector_sum_norm),B_norm)
D_c = np.cross(np.cross(C_norm,vector_sum_norm),C_norm)

len_D_a = np.linalg.norm(D_a)
len_D_b = np.linalg.norm(D_b)
len_D_c = np.linalg.norm(D_c)

D_a_norm = D_a/len_D_a
D_b_norm = D_b/len_D_b
D_c_norm = D_c/len_D_c

A1 = (np.cos(angle)*A_norm + np.sin(angle)*D_a_norm) #* len_A
B1 = (np.cos(angle)*B_norm + np.sin(angle)*D_b_norm) #* len_B
C1 = (np.cos(angle)*C_norm + np.sin(angle)*D_c_norm) #* len_C

#Unitary
new_lattice_conv = np.array([A1,B1,C1])

#Unitary
new_lattice = np.matmul(conv_matrix_inv,new_lattice_conv)
print(np.linalg.norm(new_lattice[0,:]))

new_lattice[0,:] = (new_lattice[0,:]/ np.linalg.norm(new_lattice[0,:])) * (len_A_prim)
new_lattice[1,:] = (new_lattice[1,:]/np.linalg.norm(new_lattice[1,:])) * (len_B_prim)
new_lattice[2,:] = (new_lattice[2,:]/np.linalg.norm(new_lattice[2,:]))  *(len_C_prim)
print(np.linalg.norm(new_lattice[0,:]))

aa = Structure(new_lattice,mno_pmg_opt.atomic_numbers,mno_pmg_opt.frac_coords)
Structure(np.matmul(conv_matrix,aa.lattice.matrix),[],[])

1.2352346992158365
5.31511592000219


Structure Summary
Lattice
    abc : 4.302919860799517 4.302919860792974 4.302919860792974
 angles : 89.40858791294681 89.40858791308834 89.40858791308834
 volume : 79.65642917248636
      A : 3.4951402478580436 0.0 2.5098035732434814
      B : -1.747570123931143 3.026880244427625 2.509803573239085
      C : -1.747570123931143 -3.026880244427625 2.509803573239085

In [126]:
Structure(np.matmul(conv_matrix,mno_pmg_opt.lattice.matrix),[],[])

Structure Summary
Lattice
    abc : 4.392560947738861 4.39256094773121 4.39256094773121
 angles : 90.821316026348 90.8213160265301 90.8213160265301
 volume : 84.72629516865366
      A : 3.612124378639 0.0 2.499429765535
      B : -1.8060621893189999 3.12819147352 2.499429765535
      C : -1.8060621893189999 -3.12819147352 2.499429765535

In [83]:
A

array([1.80606219, 0.        , 4.99885953])

In [58]:
mno_pmg_opt.lattice.matrix

array([[ 1.80606219,  0.        ,  4.99885953],
       [-0.90303109,  1.56409574,  4.99885953],
       [-0.90303109, -1.56409574,  4.99885953]])

In [14]:
mno_gui_opt = Crystal_gui().read_cry_gui('./isa/mno/MnO.gui')
mno_pmg_opt = cry_gui2pmg(mno_gui_opt)

a,b,c = mno_pmg_opt.lattice.abc
alpha, beta, gamma = mno_pmg_opt.lattice.angles 

directory_opt = './isa/MnO/super_6/'

#for i,disp in enumerate([0.,0.03,0.3]):
for i,disp in enumerate([0.,0.3]):
    
    #for j, angle in enumerate([0.,0.15,1.5]):
    for j, angle in enumerate([0.]):
        
        lattice = mno_pmg_opt.lattice.matrix + mno_pmg_opt.lattice.matrix*disp
        #lattice = Lattice.from_parameters(a+disp, b+disp, c+disp, alpha+angle, beta+angle, gamma+angle)
        
        #lattice = mno_pmg_opt.lattice.matrix
        
        mno_pmg = Structure(lattice,mno_pmg_opt.atomic_numbers,mno_pmg_opt.frac_coords)
        #mno_pmg_symm = SpacegroupAnalyzer(mno_pmg_tmp).get_symmetrized_structure()
        #mno_pmg = Structure(mno_pmg_symm.lattice.matrix,mno_pmg_opt.atomic_numbers,mno_pmg_symm.frac_coords)
        #display(mno_pmg_symm.lattice.matrix,mno_pmg)
        #print(mno_pmg,'\n',Structure(mno_pmg_symm.lattice.matrix,mno_pmg_opt.atomic_numbers,mno_pmg_symm.frac_coords))
        
        disp_str = get_displaced_structures(mno_pmg,atom_disp=0.01, supercell_matrix=supercell_matrix)
        mno_scel_atomic_numbers = disp_str[0].atomic_numbers

        '''norm = np.array([np.linalg.norm(mno_pmg.lattice.matrix[:,:],axis=1)]*3)
        lattice_norm = mno_pmg.lattice.matrix/norm
        new_lattice = lattice_norm-np.array([np.sum(lattice_norm,axis=0)*angle]*3)
        new_lattice_norm = new_lattice/np.array([np.linalg.norm(new_lattice[:,:],axis=1)]*3)
        new_lattice = new_lattice_norm*norm
        Structure(new_lattice,mno_pmg_opt.atomic_numbers,mno_pmg_opt.frac_coords)'''
        
        
    
        directory = './isa/MnO/super_6/%s_%s'%(i,j)
        print(i,j,directory)
        try: 
            os.mkdir(directory) 
        except OSError as error: 
            pass  
            
        range_scel = int(disp_str[0].num_sites/mno_pmg.num_sites)
        for m in range(len(disp_str)):
            for j in range(range_scel):
                disp_str[m].replace(j,1)

        spin = []
        for n in np.where(np.array(disp_str[0].atomic_numbers)==1)[0]:
            spin.extend(['%s +1'%(n+1)])
        for n in np.where(np.array(disp_str[0].atomic_numbers)==25)[0]:
            spin.extend(['%s -1'%(n+1)])

        n_spin = len(np.where(np.array(disp_str[0].atomic_numbers)==1)[0])+\
            len(np.where(np.array(disp_str[0].atomic_numbers)==25)[0])
        spin = ' '.join(spin)       

        mno_scel = cry_pmg2gui(disp_str[0],symmetry=True)
        mno_scel.atom_number = mno_scel_atomic_numbers

        write_crystal_gui(join(directory,'supercell_symm.gui'),mno_scel)

        mno_scel = cry_pmg2gui(disp_str[0],symmetry=False)
        mno_scel.atom_number = mno_scel_atomic_numbers

        write_crystal_gui(join(directory,'supercell.gui'),mno_scel)


        mno_scel_inp = Crystal_input().from_file(join(directory_opt,'MnO.d12'))


        #INPUT FILES
        mno_scel_inp = Crystal_input().from_file(join(directory_opt,'MnO.d12'))
        mno_scel_inp.scf_block.insert(-1,'ATOMSPIN\n')
        mno_scel_inp.scf_block.insert(-1,'%s\n'%n_spin)
        mno_scel_inp.scf_block.insert(-1,spin+'\n')
        mno_scel_inp.scf_block.insert(-1,'SAVEPRED\n')

        write_crystal_input(join(directory,'supercell_symm.d12'),mno_scel_inp)
        
        mno_scel_inp.scf_block.remove('SAVEPRED\n')
        
        mno_scel_inp.geom_block.insert(-1,'SYMMREMO\n')
        mno_scel_inp.scf_block.insert(-1,'GUESSYMP\n')
        mno_scel_inp.scf_block.insert(-1,'MPP\n')

        write_crystal_input(join(directory,'supercell.d12'),mno_scel_inp)
        
        mno_scel_inp.scf_block.remove('GUESSYMP\n')
        mno_scel_inp.scf_block.insert(-1,'GUESSP\n')
        
        #Displacements
        for k,displ in enumerate(disp_str[1:]):   
            mno_scel = cry_pmg2gui(displ,symmetry=False)
            #mno_scel.atom_number = mno_scel_atomic_numbers

            write_crystal_gui(join(directory,'supercell-00%s.gui'%(k+1)),mno_scel,symm=False)
            write_crystal_input(join(directory,'supercell-00%s.d12'%(k+1)),mno_scel_inp)

0 0 ./isa/MnO/super_6/0_0
1 0 ./isa/MnO/super_6/1_0


In [8]:
disp_str[0]

Structure Summary
Lattice
    abc : 13.819301392005695 13.819301392000924 13.819301392000924
 angles : 34.22791099990834 34.227910999988225 34.227910999988225
 volume : 744.5746819421284
      A : 4.695761692232 0.0 12.997034780782
      B : -2.3478808461133998 4.066648915576 12.997034780782
      C : -2.3478808461133998 -4.066648915576 12.997034780782
PeriodicSite: H (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: H (2.3479, 0.0000, 6.4985) [0.5000, 0.0000, 0.0000]
PeriodicSite: H (-1.1739, 2.0333, 6.4985) [0.0000, 0.5000, 0.0000]
PeriodicSite: H (1.1739, 2.0333, 12.9970) [0.5000, 0.5000, 0.0000]
PeriodicSite: H (-1.1739, -2.0333, 6.4985) [0.0000, 0.0000, 0.5000]
PeriodicSite: H (1.1739, -2.0333, 12.9970) [0.5000, 0.0000, 0.5000]
PeriodicSite: H (-2.3479, 0.0000, 12.9970) [0.0000, 0.5000, 0.5000]
PeriodicSite: H (0.0000, 0.0000, 19.4956) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (2.3479, -0.0000, 11.3724) [0.6250, 0.1250, 0.1250]
PeriodicSite: O (0.0000, -0.0000, 4.8739

In [107]:
matrix = SpacegroupAnalyzer(mno_pmg_opt).get_conventional_to_primitive_transformation_matrix()

In [110]:
np.round(np.matmul(np.linalg.inv(matrix),mno_pmg_opt.lattice.matrix),3)

array([[-2.709,  1.564,  0.   ],
       [ 2.709,  1.564,  0.   ],
       [ 0.   ,  0.   , 14.997]])

In [82]:
a = SpacegroupAnalyzer(mno_pmg_opt).get_symmetrized_structure()

In [83]:
a.lattice.matrix

array([[ 1.80606219,  0.        ,  4.99885953],
       [-0.90303109,  1.56409574,  4.99885953],
       [-0.90303109, -1.56409574,  4.99885953]])

In [72]:
Lattice.from_parameters(a,b,c,alpha, beta, gamma )

Lattice
    abc : 5.3151159200021905 5.315115920000354 5.315115920000355
 angles : 34.2279109999083 34.22791099998824 34.22791099998824
 volume : 42.36314758432683
      A : 2.989679431989924 0.0 4.394573259941986
      B : 1.3531190385265182 2.6659429801850294 4.3945732599446385
      C : 0.0 0.0 5.315115920000355

In [75]:
view(AseAtomsAdaptor().get_atoms(mno_pmg))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [411]:
def cart2sph(xyz):
    x = xyz[0]
    y = xyz[1]
    z = xyz[2]
    
    hxy = np.hypot(x, y)
    r = np.hypot(hxy, z)
    el = np.arctan2(z, hxy)
    az = np.arctan2(y, x)
    if np.around(az,6) ==  np.around(2*np.pi,6) \
    or np.around(az,6) ==  -np.around(2*np.pi,6):
        az = 0.
    if np.around(az,6) < 0.:
        az = np.round(2*np.pi+az,6)
    return [round(az,6), round(el,6), round(r,6)]

def sph2cart(azelr):
    az = azelr[0]
    el = azelr[1]
    r = azelr[2]
    
    x = r*np.cos(az)*np.sin(el)
    y = r*np.sin(az)*np.sin(el)
    z = r*np.cos(el)
    
    return [x,y,z]

In [428]:
#ONLY ONE
directory = './isa/MnO/test/'

range_scel = int(disp_str[0].num_sites/mno_pmg.num_sites)
for i in range(len(disp_str)):
    for j in range(range_scel):
        disp_str[i].replace(j,1)

spin = []
for i in np.where(np.array(disp_str[0].atomic_numbers)==1)[0]:
    spin.extend(['%s +1'%(i+1)])
for i in np.where(np.array(disp_str[0].atomic_numbers)==25)[0]:
    spin.extend(['%s -1'%(i+1)])

n_spin = len(np.where(np.array(disp_str[0].atomic_numbers)==1)[0])+\
    len(np.where(np.array(disp_str[0].atomic_numbers)==25)[0])
spin = ' '.join(spin)       

mno_scel = cry_pmg2gui(disp_str[0],symmetry=True)
mno_scel.atom_number = mno_scel_atomic_numbers

write_crystal_gui(join(directory,'supercell_symm.gui'),mno_scel)

mno_scel = cry_pmg2gui(disp_str[0],symmetry=False)
mno_scel.atom_number = mno_scel_atomic_numbers

write_crystal_gui(join(directory,'supercell.gui'),mno_scel)


mno_scel_inp = Crystal_input().from_file(join(directory,'MnO.d12'))


#INPUT FILES
mno_scel_inp = Crystal_input().from_file(join(directory,'MnO.d12'))
mno_scel_inp.scf_block.insert(-1,'ATOMSPIN\n')
mno_scel_inp.scf_block.insert(-1,'%s\n'%n_spin)
mno_scel_inp.scf_block.insert(-1,spin+'\n')
mno_scel_inp.scf_block.insert(-1,'SAVEPRED\n')
#mno_scel_inp.scf_block.insert(-1,'MPP\n')
mno_scel_inp.scf_block

write_crystal_input(join(directory,'supercell_symm.d12'),mno_scel_inp)

mno_scel_inp.geom_block.insert(-1,'SYMMREMO\n')
mno_scel_inp.scf_block[-3] = 'GUESSYMP\n'

write_crystal_input(join(directory,'supercell.d12'),mno_scel_inp)

mno_scel_inp.scf_block[-3] = 'GUESSP\n'

#Displacements
for i,disp in enumerate(disp_str[1:]):   
    mno_scel = cry_pmg2gui(disp,symmetry=False)
    mno_scel.atom_number = mno_scel_atomic_numbers

    write_crystal_gui(join(directory,'supercell-00%s.gui'%(i+1)),mno_scel,symm=False)
    write_crystal_input(join(directory,'supercell-00%s.d12'%(i+1)),mno_scel_inp)

## Workflow

## Aurora

In [49]:
for i,func in enumerate(['lda','gga','hybrid']):
    directory = './aurora/phonons/%s'%func
    
    opt_input = Crystal_input().from_file(join(directory,'geom_opt.d12'))
    
    n_disp = 2
    
    shutil.copy(join(directory,'supercell.ext'),join(directory,'supercell_symm.gui'))
    shutil.copy(join(directory,'supercell.ext'),join(directory,'supercell.gui'))

    supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell_symm.gui'))
    supercell_pmg = cry_gui2pmg(supercell_gui)
    supercell_gui = cry_pmg2gui(supercell_pmg)
    write_crystal_gui(join(directory,'supercell_symm.gui'),supercell_gui)

    symm_inp = Crystal_input().from_file('./aurora/phonons/supercell_symm.d12')
    symm_inp.func_block = opt_input.func_block
    write_crystal_input(join(directory,'supercell_symm.d12'),symm_inp)

    no_symm = Crystal_input().from_file('./aurora/phonons/supercell_nosymm.d12')
    no_symm.func_block = opt_input.func_block
    write_crystal_input(join(directory,'supercell.d12'),no_symm)

    disp_inp = Crystal_input().from_file('./aurora/phonons/supercell_disp.d12')
    disp_inp.func_block = opt_input.func_block

    for i in range(1,n_disp+1):

        #disp_inp = Crystal_input().from_file('./aurora/phonons/supercell_disp.d12')

        shutil.copy(join(directory,'supercell-00%s.ext'%i),join(directory,'supercell-00%s.gui'%i))
        #supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell-00%s.gui'%i))
        #supercell_pmg = cry_gui2pmg(supercell_gui)
        #supercell_gui = cry_pmg2gui(supercell_pmg)

        #write_crystal_gui(join(directory,'supercell-00%s.gui'%i),supercell_gui)
        write_crystal_input(join(directory,'supercell-00%s.d12'%i),disp_inp)

In [46]:
directory = './aurora/phonons/test'

opt_input = Crystal_input().from_file(join(directory,'geom_opt.d12'))

n_disp = 2

shutil.copy(join(directory,'supercell.ext'),join(directory,'supercell_symm.gui'))
shutil.copy(join(directory,'supercell.ext'),join(directory,'supercell.gui'))

supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell_symm.gui'))
supercell_pmg = cry_gui2pmg(supercell_gui)
supercell_gui = cry_pmg2gui(supercell_pmg)
write_crystal_gui(join(directory,'supercell_symm.gui'),supercell_gui)

symm_inp = Crystal_input().from_file('./aurora/phonons/supercell_symm.d12')
symm_inp.func_block = opt_input.func_block
write_crystal_input(join(directory,'supercell_symm.d12'),symm_inp)

no_symm = Crystal_input().from_file('./aurora/phonons/supercell_nosymm.d12')
no_symm.func_block = opt_input.func_block
write_crystal_input(join(directory,'supercell.d12'),no_symm)

disp_inp = Crystal_input().from_file('./aurora/phonons/supercell_disp.d12')
disp_inp.func_block = opt_input.func_block

for i in range(1,n_disp+1):

    #disp_inp = Crystal_input().from_file('./aurora/phonons/supercell_disp.d12')

    shutil.copy(join(directory,'supercell-00%s.ext'%i),join(directory,'supercell-00%s.gui'%i))
    #supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell-00%s.gui'%i))
    #supercell_pmg = cry_gui2pmg(supercell_gui)
    #supercell_gui = cry_pmg2gui(supercell_pmg)

    #write_crystal_gui(join(directory,'supercell-00%s.gui'%i),supercell_gui)
    write_crystal_input(join(directory,'supercell-00%s.d12'%i),disp_inp)

In [17]:
directory = './aurora/phonons/lda'
supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell_symm.gui'))
supercell_pmg = cry_gui2pmg(supercell_gui)

In [19]:
a = SpacegroupAnalyzer(supercell_pmg).get_symmetry_operations()

In [43]:
I = np.identity(3,dtype=int)*-1
for op in a:
    if np.sum(np.array(op.rotation_matrix,dtype=int) == I) == 9:
        print('yes')
        


In [10]:
symm_inp.func_block

['a']

In [38]:
directory = './aurora/phonons/lda/'
n_disp = 2

for i in range(1,n_disp+1):
    
    disp_inp = Crystal_input().from_file('./aurora/phonons/supercell_disp.d12')
    
    shutil.copy(join(directory,'supercell-00%s.ext'%i),join(directory,'supercell-00%s.gui'%i))
    supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell-00%s.gui'%i))
    supercell_pmg = cry_gui2pmg(supercell_gui)
    supercell_gui = cry_pmg2gui(supercell_pmg)
    
    write_crystal_gui(join(directory,'supercell-00%s.gui'%i),supercell_gui)
    write_crystal_input(join(directory,'supercell-00%s.gui'%i),disp_inp)
    


<class 'crystal_functions.file_readwrite.Crystal_gui'>
<class 'crystal_functions.file_readwrite.Crystal_gui'>


In [51]:
cry_gui = Crystal_gui().read_cry_gui('./mgo.gui')
mgo_pmg = cry_gui2pmg(cry_gui)
mgo_pmg

Structure Summary
Lattice
    abc : 2.9818692962636706 2.9818692962636706 2.9818692962636706
 angles : 59.99999999999999 59.99999999999999 59.99999999999999
 volume : 18.747821578249994
      A : 0.0 2.1085 2.1085
      B : 2.1085 0.0 2.1085
      C : 2.1085 2.1085 0.0
PeriodicSite: Mg (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: O (0.0000, 0.0000, -2.1085) [-0.5000, -0.5000, 0.5000]

In [52]:
#mgo_pmg.make_supercell(np.identity(3)*2)
mgo_pmg.make_supercell([[-1,1,1],[1,-1,1],[1,1,-1]])
mgo_pmg

Structure Summary
Lattice
    abc : 4.217 4.217 4.217
 angles : 90.0 90.0 90.0
 volume : 74.99128631299997
      A : 4.217 0.0 0.0
      B : 0.0 4.217 0.0
      C : 0.0 0.0 4.217
PeriodicSite: Mg (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Mg (2.1085, 2.1085, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Mg (2.1085, 0.0000, 2.1085) [0.5000, 0.0000, 0.5000]
PeriodicSite: Mg (0.0000, 2.1085, 2.1085) [0.0000, 0.5000, 0.5000]
PeriodicSite: O (0.0000, 0.0000, 2.1085) [0.0000, 0.0000, 0.5000]
PeriodicSite: O (2.1085, 2.1085, 2.1085) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (2.1085, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: O (0.0000, 2.1085, 0.0000) [0.0000, 0.5000, 0.0000]

In [54]:
mgo_pmg.translate_sites(1,[0.,0.,0.003])

In [55]:
len(SpacegroupAnalyzer(mgo_pmg).get_symmetry_operations())
#mgo_gui = cry_pmg2gui(mgo_pmg)

8

In [83]:
write_crystal_gui('./MnO_initial.gui',mgo_gui)

In [88]:
mno_gui = Crystal_gui().read_cry_gui('./MnO_initial.gui')
mno_pmg = cry_gui2pmg(mno_gui)

In [89]:
mno_pmg.replace(2,'H')

In [90]:
mno_pmg.make_supercell(np.identity(3)*2)

In [91]:
mno_pmg

Structure Summary
Lattice
    abc : 10.630231840004381 10.63023184000071 10.63023184000071
 angles : 34.22791099990832 34.22791099998824 34.22791099998824
 volume : 338.9051806746147
      A : 3.61212437864 0.0 9.99771906214
      B : -1.806062189318 3.12819147352 9.99771906214
      C : -1.806062189318 -3.12819147352 9.99771906214
PeriodicSite: Mn (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Mn (2.7091, -1.5641, 14.9966) [1.0000, 0.0000, 0.5000]
PeriodicSite: Mn (-0.9030, 1.5641, 4.9989) [0.0000, 0.5000, 0.0000]
PeriodicSite: Mn (1.8061, -0.0000, 19.9954) [1.0000, 0.5000, 0.5000]
PeriodicSite: Mn (1.8061, 0.0000, 4.9989) [0.5000, 0.0000, 0.0000]
PeriodicSite: Mn (0.9030, -1.5641, 9.9977) [0.5000, 0.0000, 0.5000]
PeriodicSite: Mn (0.9030, 1.5641, 9.9977) [0.5000, 0.5000, 0.0000]
PeriodicSite: Mn (0.0000, -0.0000, 14.9966) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (1.8061, 0.0000, 8.7480) [0.6250, 0.1250, 0.1250]
PeriodicSite: O (0.9030, -1.5641, 13.7469) [0.6250, 0.12

In [82]:
for i in np.where(np.array(mno_pmg.atomic_numbers)==25)[0]:
    print(i,' +1')
    #mno_pmg.replace([16, 17, 18, 19, 20, 21, 22, 23],['Mn']*8)

0  +1
1  +1
2  +1
3  +1
4  +1
5  +1
6  +1
7  +1
8  +1
9  +1
10  +1
11  +1
12  +1
13  +1
14  +1
15  +1
24  +1
25  +1
26  +1
27  +1
28  +1
29  +1
30  +1
31  +1
32  +1
33  +1
34  +1
35  +1
36  +1
37  +1
38  +1
39  +1
40  +1
41  +1
42  +1
43  +1
44  +1
45  +1
46  +1
47  +1
48  +1
49  +1
50  +1
51  +1
52  +1
53  +1
54  +1
55  +1
56  +1
57  +1
58  +1
59  +1
60  +1
61  +1
62  +1
63  +1
128  +1
129  +1
130  +1
131  +1
132  +1
133  +1
134  +1
135  +1
136  +1
137  +1
138  +1
139  +1
140  +1
141  +1
142  +1
143  +1
144  +1
145  +1
146  +1
147  +1
148  +1
149  +1
150  +1
151  +1
152  +1
153  +1
154  +1
155  +1
156  +1
157  +1
158  +1
159  +1
160  +1
161  +1
162  +1
163  +1
164  +1
165  +1
166  +1
167  +1
168  +1
169  +1
170  +1
171  +1
172  +1
173  +1
174  +1
175  +1
176  +1
177  +1
178  +1
179  +1
180  +1
181  +1
182  +1
183  +1
184  +1
185  +1
186  +1
187  +1
188  +1
189  +1
190  +1
191  +1


In [94]:
mno_pmg.translate_sites(0,[0.,0.,0.003])

In [74]:
view(AseAtomsAdaptor().get_atoms(cry_gui2pmg(Crystal_gui().read_cry_gui('./isa/MnO_super.out'))))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



EXITING: a .gui file needs to be specified
Traceback (most recent call last):
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/crystal_functions/file_readwrite.py", line 1957, in read_cry_gui
    file = open(gui_file, 'r')
FileNotFoundError: [Errno 2] No such file or directory: './isa/MnO_super.out.gui'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_62530/1747659824.py", line 1, in <module>
    view(AseAtomsAdaptor().get_atoms(cry_gui2pmg(Crystal_gui().read_cry_gui('./isa/MnO_super.out'))))
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/crystal_functions/file_readwrite.py", line 1962, in read_cry_gui
    sys.exi

TypeError: object of type 'NoneType' has no len()

In [78]:
view(AseAtomsAdaptor().get_atoms(cry_out2pmg(Crystal_output().read_cry_output('./isa/MnO_super.out'))))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [93]:
mno_prim = Structure([[0.0, 2.657557960000818, 2.657557960000818],
                      [2.657557960000818, 0.0, 2.657557960000818],
                      [2.657557960000818, 2.657557960000818, 0.0]],
                     ['Mn','O'],[[0.,0.,0.],[.5,.5,.5]])
mno_conv = SpacegroupAnalyzer(mno_prim).get_conventional_standard_structure()
mno_conv.replace(0,1)
mno_conv.replace(2,1)
mno_prim_symm = SpacegroupAnalyzer(mno_conv).get_primitive_standard_structure()
#a = np.array(mno_prim_symm.equivalent_indices)[:,1:].flatten()
#mno_prim_symm.replace(1,25)
mno_prim_symm.make_supercell(np.identity(3)*2)
write_crystal_gui('mno_super_symm.gui',cry_pmg2gui(mno_prim_symm))

In [70]:
SpacegroupAnalyzer(mno_prim_symm).get_symmetrized_structure()

SymmetrizedStructure
Full Formula (Mn8 H8 O16)
Reduced Formula: MnHO2
Spacegroup: P4/mmm (123)
abc   :   5.963739   5.963739   8.434000
angles:  90.000000  90.000000  90.000000
Sites (32)
  #  SP       a     b     c  Wyckoff
---  ----  ----  ----  ----  ---------
  0  Mn    0.25  0.25  0.25  8d
  1  H     0     0     0     8a
  2  O     0.25  0.25  0     8c
  3  O     0     0     0.25  8b

In [84]:
mno_conv = SpacegroupAnalyzer(mno_prim).get_primitive_standard_structure()

In [85]:
mno_conv

Structure Summary
Lattice
    abc : 2.9818692962636706 2.9818692962636706 2.9818692962636706
 angles : 59.99999999999999 59.99999999999999 59.99999999999999
 volume : 18.747821578249994
      A : -1.2910838880010971e-16 2.1085 2.1085
      B : 2.1085 0.0 2.1085
      C : 2.1085 2.1085 2.5821677760021943e-16
PeriodicSite: Mn (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: O (2.1085, 2.1085, 2.1085) [0.5000, 0.5000, 0.5000]

In [8]:
phonopy_gui = Crystal_gui().read_cry_gui('./aurora/phonons/supercell-001.gui')
len(SpacegroupAnalyzer(cry_gui2pmg(phonopy_gui)).get_symmetry_operations())

192

In [9]:
write_crystal_gui('./aurora/phonons/supercell-001.gui',cry_pmg2gui(cry_gui2pmg(phonopy_gui)))

In [20]:
aurora_out = Crystal_output().read_cry_output('./aurora/02_LDA/02_GEOM_OPT_LDA/GaP.out')
aurora_out.get_band_gap()

1.6899

In [22]:
aurora_out = Crystal_output().read_cry_output('./aurora/03_GaP_Crystal/GGA/Optimisation/GaP_GEOMETRY_OPT.out')
aurora_out.get_band_gap()

1.7288

In [15]:
SpacegroupAnalyzer(cry_out2pmg(aurora_out)).get_primitive_standard_structure()

Structure Summary
Lattice
    abc : 3.833386195705131 3.833386195705131 3.8333861957051303
 angles : 119.99999999999999 119.99999999999999 90.0
 volume : 39.83205619303444
      A : -1.9166930978525656 1.9166930978525654 2.71061337389
      B : 1.9166930978525656 -1.9166930978525654 2.71061337389
      C : 1.9166930978525651 1.9166930978525654 -2.7106133738899993
PeriodicSite: Ga (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: P (0.0000, -0.0000, 4.0659) [0.7500, 0.7500, 0.0000]

In [17]:
aurora_out = Crystal_output().read_cry_output('./aurora/03_GaP_Crystal/Hybrid/01_Opti/GaP_GEOMETRY_OPT.out')
aurora_out.get_band_gap()

3.0434

In [12]:
aurora_out.get_final_energy()

-61620.78437587766

In [13]:
SpacegroupAnalyzer(cry_out2pmg(aurora_out)).get_primitive_standard_structure()

Structure Summary
Lattice
    abc : 3.833386195705131 3.833386195705131 3.8333861957051303
 angles : 119.99999999999999 119.99999999999999 90.0
 volume : 39.83205619303444
      A : -1.9166930978525656 1.9166930978525654 2.71061337389
      B : 1.9166930978525656 -1.9166930978525654 2.71061337389
      C : 1.9166930978525651 1.9166930978525654 -2.7106133738899993
PeriodicSite: Ga (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: P (0.0000, -0.0000, 4.0659) [0.7500, 0.7500, 0.0000]

In [25]:
cry_bands = Properties_input().make_bands_block()

TypeError: make_bands_block() missing 4 required positional arguments: 'k_path', 'n_kpoints', 'first_band', and 'last_band'

In [26]:
from pymatgen.symmetry.bandstructure import HighSymmKpath
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from crystal_functions.file_readwrite import write_properties_input
from crystal_functions.file_readwrite import Crystal_output
from crystal_functions.file_readwrite import Properties_input
from crystal_functions.convert import cry_out2pmg


#Create the bands input object
bands_input = Properties_input()
#Add the newk block to the input object
bands_input.make_newk_block(12,24)

#Read the structure
mgo = Crystal_output().read_cry_output('aurora/GaP.out')
mgo = cry_out2pmg(mgo)
mgo_prim = SpacegroupAnalyzer(mgo).get_primitive_standard_structure(international_monoclinic=False)

#Obtain the k path object
k_path = HighSymmKpath(mgo_prim)

n_kpoints = 200
first_band = 1
last_band = 26
bands_input.make_bands_block(k_path,n_kpoints,first_band,last_band)

#Write the input
write_properties_input('aurora/GaP.d3',bands_input)